In [6]:
import numpy as np
import pandas as pd
import json 

In [8]:
#Install beautiful soup
#!conda install beautifulsoup4
#!conda install lxml
#!conda install html5lib
#!conda install requests
#!conda install -c conda-forge folium=0.5.0 --yes 
print('APIs installed.')

APIs installed.


In [15]:
from bs4 import BeautifulSoup
import requests
wikiweb_link="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source=requests.get(wikiweb_link).text

soup=BeautifulSoup(source,'lxml')

citytable=soup.find('table',class_="wikitable sortable")
dfcity = pd.read_html(str(citytable), header=0)[0]

dfcity['Postcode'] = dfcity['Postcode'].astype(str)
dfcity['Borough'] = dfcity['Borough'].astype(str)
dfcity['Neighbourhood'] = dfcity['Neighbourhood'].astype(str)
dfcity.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [24]:
dfcity=dfcity.drop(dfcity[dfcity.Borough=='Not assigned'].index)
dfcity=dfcity.groupby(['Postcode',"Borough"])["Neighbourhood"].apply(lambda x: ','.join(x)).reset_index()

dfcity['Neighbourhood'].loc[dfcity['Neighbourhood'].str.contains('Not assigned')]=dfcity['Borough']

dfcity.shape

(103, 3)

In [25]:
#!wget -O Geospatial_Coordinates.csv http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv
print ("csv fetched")

csv fetched


In [37]:
coorsourcedp = pd.read_csv("Geospatial_Coordinates.csv")

column_names=['Postcode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']

neighborhoods = pd.DataFrame(columns=column_names)

neighborhoods['Latitude'] = neighborhoods['Latitude'].astype(np.float64)
neighborhoods['Longitude'] = neighborhoods['Longitude'].astype(np.float64)

tlat=0.0
tlong=0.0

for data in dfcity.itertuples():
    dummydf=coorsourcedp[coorsourcedp['Postal Code']==data[1]]   
    tlat=dummydf['Latitude'].values[0]
    tlong=dummydf['Longitude'].values[0]
    neighborhoods = neighborhoods.append({'Postcode': data[1],
                                        'Borough': data[2],
                                        'Neighborhood': data[3],
                                        'Latitude': tlat,
                                        'Longitude': tlong}, ignore_index=True)

In [38]:
neighborhoods.head(10)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


In [42]:
import folium

ImportError: No module named 'folium'

In [55]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


In [60]:
from geopy.geocoders import Nominatim
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 43.653963, -79.387207.


In [63]:
torontoneighs= neighborhoods[neighborhoods['Borough'].str.contains('Toronto')].reset_index(drop=True)
torontoneighs.head()
torontoneighs.shape

(38, 5)

In [70]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, torontoneigh in zip(torontoneighs['Latitude'], torontoneighs['Longitude'], torontoneighs['Neighborhood'],torontoneighs['Borough']):
    label = '{}, {}'.format(torontoneigh, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
 
map_toronto

NameError: name 'folium' is not defined

In [77]:
import json 
import requests 
from pandas.io.json import json_normalize 

CLIENT_ID = '43T434JZQOA3SSDEKRFGYOSXOFH3DAJBFHU0MWEBLHTM0DVB' 
CLIENT_SECRET = 'SOFPDSOSEPSHU5AV2VVPFDV5QEQMBRQM5KQT2FORBP2XH2IV' 
VERSION = '20180605'
radius=500
LIMIT=100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 43T434JZQOA3SSDEKRFGYOSXOFH3DAJBFHU0MWEBLHTM0DVB
CLIENT_SECRET:SOFPDSOSEPSHU5AV2VVPFDV5QEQMBRQM5KQT2FORBP2XH2IV


In [81]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [82]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [85]:
toronto_venues = getNearbyVenues(names=torontoneighs['Neighborhood'],
                                   latitudes=torontoneighs['Latitude'],
                                   longitudes=torontoneighs['Longitude']
                                  )
print(toronto_venues.shape)

The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvall

In [86]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 240 uniques categories.


In [95]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.rename(columns={'Neighborhood': 'Neigh'},inplace=True)

toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
#toronto_onehot.head()

In [97]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

In [101]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                 venue  freq
0          Coffee Shop  0.06
1           Steakhouse  0.04
2      Thai Restaurant  0.04
3  American Restaurant  0.04
4                 Café  0.04


----Berczy Park----
          venue  freq
0   Coffee Shop  0.07
1  Cocktail Bar  0.05
2        Bakery  0.04
3    Steakhouse  0.04
4   Cheese Shop  0.04


----Brockton,Exhibition Place,Parkdale Village----
            venue  freq
0  Breakfast Spot  0.09
1            Café  0.09
2     Coffee Shop  0.09
3   Grocery Store  0.05
4   Burrito Place  0.05


----Business Reply Mail Processing Centre 969 Eastern----
                  venue  freq
0            Comic Shop  0.06
1            Restaurant  0.06
2                   Spa  0.06
3  Fast Food Restaurant  0.06
4            Smoke Shop  0.06


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
              venue  freq
0   Airport Service  0.20
1    Airport Lounge  0.13
2  Airport Term

In [108]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [115]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,American Restaurant,Steakhouse,Thai Restaurant,Café,Bar,Restaurant,Asian Restaurant,Bakery,Burger Joint
1,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Cheese Shop,Steakhouse,Seafood Restaurant,Farmers Market,Café,Pub,Bakery
2,"Brockton,Exhibition Place,Parkdale Village",Breakfast Spot,Café,Coffee Shop,Yoga Studio,Bar,Burrito Place,Restaurant,Caribbean Restaurant,Climbing Gym,Pet Store
3,Business Reply Mail Processing Centre 969 Eastern,Gym / Fitness Center,Recording Studio,Skate Park,Park,Spa,Brewery,Farmers Market,Fast Food Restaurant,Light Rail Station,Burrito Place
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Boat or Ferry,Sculpture Garden,Boutique,Airport Gate,Airport,Plane
5,"Cabbagetown,St. James Town",Coffee Shop,Restaurant,Café,Pizza Place,Italian Restaurant,Pub,Bakery,Playground,Sandwich Place,Japanese Restaurant
6,Central Bay Street,Coffee Shop,Italian Restaurant,Bar,Bubble Tea Shop,Burger Joint,Café,Ice Cream Shop,Chinese Restaurant,Spa,Thai Restaurant
7,"Chinatown,Grange Park,Kensington Market",Café,Bar,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Dumpling Restaurant,Mexican Restaurant,Bakery,Coffee Shop,Chinese Restaurant,Grocery Store
8,Christie,Café,Grocery Store,Park,Italian Restaurant,Athletics & Sports,Diner,Restaurant,Nightclub,Baby Store,Coffee Shop
9,Church and Wellesley,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Gay Bar,Restaurant,Café,Fast Food Restaurant,Gym,Pub,Bubble Tea Shop


In [120]:
from sklearn.cluster import KMeans

kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
toronto_grouped_clustering

kmeans = KMeans(init="random", n_clusters=kclusters, n_init=12)
kmeans=kmeans.fit(toronto_grouped_clustering)


kmeans.labels_


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 4, 0, 0, 0, 0, 2,
       0, 3, 0, 0, 4, 3, 0, 0, 0, 0, 0, 0, 1, 0, 0], dtype=int32)

In [135]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = torontoneighs

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

#toronto_merged.head()

ValueError: cannot insert Cluster Labels, already exists

In [126]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [134]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

NameError: name 'folium' is not defined

In [143]:
#First Cluster
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East Toronto,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Furniture / Home Store,Bookstore,Italian Restaurant,Spa,Brewery,Bubble Tea Shop,Restaurant
2,East Toronto,0,Pizza Place,Ice Cream Shop,Pet Store,Pub,Movie Theater,Sandwich Place,Burrito Place,Burger Joint,Brewery,Park
3,East Toronto,0,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Bar,Stationery Store,Fish Market,Latin American Restaurant,Coworking Space
5,Central Toronto,0,Clothing Store,Gym,Asian Restaurant,Breakfast Spot,Park,Hotel,Sandwich Place,Dance Studio,Burger Joint,Food & Drink Shop
6,Central Toronto,0,Coffee Shop,Sporting Goods Shop,Clothing Store,Fast Food Restaurant,Metro Station,Mexican Restaurant,Diner,Dessert Shop,Park,Chinese Restaurant
7,Central Toronto,0,Dessert Shop,Sandwich Place,Pizza Place,Italian Restaurant,Café,Coffee Shop,Sushi Restaurant,Park,Seafood Restaurant,Burger Joint
11,Downtown Toronto,0,Coffee Shop,Restaurant,Café,Pizza Place,Italian Restaurant,Pub,Bakery,Playground,Sandwich Place,Japanese Restaurant
12,Downtown Toronto,0,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Gay Bar,Restaurant,Café,Fast Food Restaurant,Gym,Pub,Bubble Tea Shop
13,Downtown Toronto,0,Coffee Shop,Bakery,Pub,Park,Café,Restaurant,Breakfast Spot,Mexican Restaurant,Theater,Yoga Studio
14,Downtown Toronto,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Bar,Restaurant,Diner,Bookstore,Hotel


In [144]:
#Second Cluster
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,1,Health Food Store,Neigh,Coffee Shop,Pub,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
9,Central Toronto,1,Coffee Shop,Pub,Fried Chicken Joint,Supermarket,Pizza Place,Vietnamese Restaurant,Bagel Shop,Convenience Store,Light Rail Station,Sushi Restaurant


In [145]:
#Third Cluster
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,3,Playground,Tennis Court,Department Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
22,Central Toronto,3,Health & Beauty Service,Garden,Fast Food Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


In [146]:
#Fourth Cluster
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Downtown Toronto,4,Park,Playground,Trail,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
23,Central Toronto,4,Trail,Jewelry Store,Park,Sushi Restaurant,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Yoga Studio
